In [2]:
# 1. Clone della repository e installazione delle librerie
!git clone https://github.com/leonardoCosta02/Hawk-AI-CV-Project.git
# Esempio: !git clone https://github.com/GruppoCV/Hawk-AI-CV-Project.git

# Entra nella cartella di progetto e aggiungila al PATH di Python
import os
repo_name = "Hawk-AI-CV-Project" # Sostituisci con il nome del tuo repository
os.chdir(repo_name)

import sys
sys.path.insert(0, os.getcwd())

# Installa OpenCV (se non è già presente, Colab la ha) e Matplotlib
!pip install numpy opencv-python matplotlib

print("Setup completato. Sei nella cartella:", os.getcwd())

Cloning into 'Hawk-AI-CV-Project'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 28 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (28/28), 10.32 KiB | 330.00 KiB/s, done.
Resolving deltas: 100% (3/3), done.
Setup completato. Sei nella cartella: /content/Hawk-AI-CV-Project
